In [166]:
import os
import numpy as np
import pandas as pd
import json

%env GOOGLE_APPLICATION_CREDENTIALS "FoodCampus-f38ced1dcb50.json"
%env GCLOUD_PROJECT "foodcampus"

env: GOOGLE_APPLICATION_CREDENTIALS="FoodCampus-f38ced1dcb50.json"
env: GCLOUD_PROJECT="foodcampus"


In [167]:
from google.cloud import firestore

# Project ID is determined by the GCLOUD_PROJECT environment variable
db = firestore.Client()

DefaultCredentialsError: File "FoodCampus-f38ced1dcb50.json" was not found.

In [154]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

cred=credentials.Certificate('/Users/nathanwong/Desktop/FoodCampus-f38ced1dcb50.json')
firebase_admin.initialize_app(cred, {
    'storageBucket': 'foodcampus.appspot.com'
})
db = firestore.client()
bucket = storage.bucket()
blob = bucket.blob('hello.txt')
outfile='./hello.txt'
blob.upload_from_filename(outfile)

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [26]:
db = firestore.client()
bucket = storage.bucket()
blob = bucket.blob('Practice/hello.txt')
outfile='./hello.txt'
blob.upload_from_filename(outfile)

In [82]:
import csv
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore

def batch_data(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def main():
    # Init Firebase
    store = firestore.client()
    collection_name = "restaurants"

    # Init Parsing
    data = []
    headers = []
    file_path = os.path.join("./Restaurant Data", "restaurants_menu.csv")

    # Parse
    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                for header in row:
                    headers.append(header)
                line_count += 1
            else:
                obj = {}
                for idx, item in enumerate(row):
                    obj[headers[idx]] = item
                data.append(obj)
                line_count += 1
        print('Processed {line_count} lines.')

    for batched_data in batch_data(data, 499):
        batch = store.batch()
        for data_item in batched_data:
            doc_ref = store.collection(collection_name).document(data_item['\ufeffname'])
            sub_tree = doc_ref.collection("Items").document(data_item['item'])
            print(data_item, doc_ref)
            batch.set(sub_tree, data_item)
        batch.commit()

    print('Done')

In [84]:
main()

Processed {line_count} lines.
{'\ufeffname': "Gypsy's Trattoria Italiana", 'item': 'Crazy Alfredo', 'cash_price': '10.49', 'credit_price': '10.99', 'popularity': '8', 'options': ''} <google.cloud.firestore_v1.document.DocumentReference object at 0x10c202c18>
{'\ufeffname': "Gypsy's Trattoria Italiana", 'item': 'Melanzana Con Pollo', 'cash_price': '10.49', 'credit_price': '10.99', 'popularity': '9', 'options': ''} <google.cloud.firestore_v1.document.DocumentReference object at 0x10c202d68>
{'\ufeffname': "Gypsy's Trattoria Italiana", 'item': "Godfather's favorite", 'cash_price': '10.49', 'credit_price': '10.99', 'popularity': '10', 'options': ''} <google.cloud.firestore_v1.document.DocumentReference object at 0x10c202dd8>
{'\ufeffname': "Gypsy's Trattoria Italiana", 'item': "Gypsy's Chicken", 'cash_price': '10.49', 'credit_price': '10.99', 'popularity': '8', 'options': ''} <google.cloud.firestore_v1.document.DocumentReference object at 0x10c202e48>
{'\ufeffname': "Gypsy's Trattoria Ital

Done


In [150]:
doc_ref = db.collection('restaurants').document('Chipotle')

try:
    doc = doc_ref.get()
    print(u'Document data: {}'.format(doc.to_dict()))
except google.cloud.exceptions.NotFound:
    print(u'No such document!')


Document data: None


In [147]:
users_ref = db.collection('restaurants')
docs = users_ref.stream()

for doc in docs:
    print(u'{} => {}'.format(doc.id, doc.to_dict()))

In [151]:
db = firestore.client()

DefaultCredentialsError: File "/Users/nathanwong/Desktop/FoodCampus-f38ced1dcb50.json" was not found.

In [138]:
a_lovelace_ref = db.document(u'restaurants/Chipotle')

In [144]:
a_lovelace_ref.get().to_dict()